In [ ]:
#Preprocessing Imports
import pandas as pd
import numpy as np

In [ ]:
# drug, _, label

def concatenate_label_data(df1, df2, labeldf):
    df1_T = df1.T
    df2_T = df2.T
    
    X = []
    y = np.array(labeldf.iloc[:, -1:].values)
    
    for index, row in labeldf.iterrows():
        drug_name = row[0]
        vector_name = row[1]

        # Fetch the corresponding embeddings
        drug_embedding = df1_T.loc[drug_name].values
        vector_embedding = df2_T.loc[vector_name].values

        # Concatenate the embeddings
        concatenated_embedding = np.concatenate((drug_embedding, vector_embedding))

        # Add the concatenated embedding to the feature list
        X.append(concatenated_embedding)

    # Convert the list of feature vectors into a numpy array for further processing
    X = np.array(X)

    # Check the shapes of X and y to ensure they are as expected
    print(X.shape, y.shape)
    return X, y

In [ ]:
#replace with path to data_InSummary
path = '../PM 1/data_InSummary/Drug_{}/{}/drug_embedding.csv'

response_2_path = '../PM 1/data_InSummary/Drug_response/{}/ccl_feature.csv'
response_label_path = '../PM 1/data_InSummary/Drug_response/{}/response.csv'

se_2_path = '../PM 1/data_InSummary/Drug_se/disease_embedding.csv'
se_label_path = '../PM 1/data_InSummary/Drug_se/{}/data_afterSampling.csv'

target_path = '../PM 1/data_InSummary/Drug_target/{}/data_sampled.parquet'


ccle_1 = pd.read_csv(path.format("response", "ccle"))
ccle_2 = pd.read_csv(response_2_path.format("ccle"))
ccle_labels = pd.read_csv(response_label_path.format("ccle"))
gdsc_1 = pd.read_csv(path.format("response", "gdsc"))
gdsc_2 = pd.read_csv(response_2_path.format("gdsc")).drop(['Unnamed: 0'], axis=1)
gdsc_labels = pd.read_csv(response_label_path.format("gdsc"))
pdx_1 = pd.read_csv(path.format("response", "pdx"))
pdx_2 = pd.read_csv(response_2_path.format("pdx")).drop(['Unnamed: 0'], axis=1)
pdx_labels = pd.read_csv(response_label_path.format("pdx"))

offside = pd.read_csv(path.format("se", "offside"))
offside_labels = pd.read_csv(se_label_path.format("offside"))
sider = pd.read_csv(path.format("se", "sider"))
sider_labels = pd.read_csv(se_label_path.format("sider"))
se_2 = pd.read_csv(se_2_path)

In [ ]:
X = []
y = []
X_i, y_i = concatenate_label_data(ccle_1, ccle_2, ccle_labels)
X.append(X_i)
y.append(y_i)
X_i, y_i = concatenate_label_data(gdsc_1, gdsc_2, gdsc_labels)
X.append(X_i)
y.append(y_i)
X_i, y_i = concatenate_label_data(pdx_1, pdx_2, pdx_labels)
X.append(X_i)
y.append(y_i)
X_i, y_i = concatenate_label_data(offside, se_2, offside_labels)
X.append(X_i)
y.append(y_i)
X_i, y_i = concatenate_label_data(sider, se_2, sider_labels)
X.append(X_i)
y.append(y_i)
print(len(X), len(y))

In [ ]:
drugbank = pd.read_parquet(target_path.format("drugbank"))
repurposing_hub = pd.read_parquet(target_path.format("repurposing_hub"))
stitch = pd.read_parquet(target_path.format("stitch"))

In [ ]:
X_i = drugbank.iloc[:, 2:-1].values
y_i = np.array(drugbank.iloc[:,-1:].values)
X.append(X_i)
y.append(y_i)
X_i = repurposing_hub.iloc[:, 2:-1].values
y_i = np.array(repurposing_hub.iloc[:,-1:].values)
X.append(X_i)
y.append(y_i)
X_i = stitch.iloc[:, :-3].values
y_i = np.array(stitch.iloc[:, -3:-2].values)
X.append(X_i)
y.append(y_i)

print(len(X), len(y))

In [ ]:
from sklearn.decomposition import PCA

filenames = ["drugbank", "repurposing_hub", "stitch"]
#filenames = ["ccle", "gdsc", "pdx", "offside", "sider"] 

for i in range(len(X)):
    pca = PCA(n_components=50)
    X_new = pca.fit_transform(X[i])
    data = np.append(X_new, y[i], axis=1)
    np.save(f'{filenames[i]}.npy', data)